In [40]:
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import datetime

In [41]:
def obtener_contenido_pagina(url):
    respone=requests.get(url)
    return respone.text

def analizar_contenido_html(html):
    return BeautifulSoup(html,'html.parser')

In [42]:
data=[]
def procesar_pagina(soup):
    titulos=[]
    tomatoers=[] 
    audience_scores=[]
    estrenos=[]

    #titulos
    titulo_items=soup.find_all('span',{'data-qa': 'discovery-media-list-item-title'})
    
    for item in titulo_items:
        titulo=item.text.strip()
        titulos.append(titulo)

    #tomatoers
    tomatoers_items=soup.find_all('score-pairs')
    
    for item in tomatoers_items:
        valor_critic=item['criticsscore']
        tomatoers.append(valor_critic)

    tomatoers= [None if item == "" else item for item in tomatoers]
    
    #audience
    audience_score=soup.find_all('score-pairs')

    for item in audience_score:
        valor_audience=item['audiencescore']
        audience_scores.append(valor_audience)
    
    audience_scores= [None if item == "" else item for item in audience_scores]

    #estrenos
    estrenos_items=soup.find_all('span',{'data-qa':'discovery-media-list-item-start-date'})

    for item in estrenos_items:
        estreno=item.text.strip()
        estrenos.append(estreno)

    estrenos=[item.replace('Opened ', '') for item in estrenos]
    estrenos=[item.replace('Opens ', '') for item in estrenos]
    
    min_length = min(len(titulos), len(tomatoers), len(audience_scores), len(estrenos))

    for i in range(min_length):
        data.append({
            "Titles": titulos[i] if i < len(titulos) else None,
            "Critics Score": tomatoers[i]  if i < len(tomatoers) else None,
            "Audience Score": audience_scores[i]  if i < len(audience_scores) else None,
            "Premiere": estrenos[i] if i < len(estrenos) else None,
        })



In [43]:
url= "https://www.rottentomatoes.com/browse/movies_in_theaters/?page=4"
contenido_pagina = obtener_contenido_pagina(url)
soup = analizar_contenido_html(contenido_pagina)
procesar_pagina(soup)


In [44]:
import pandas as pd

df = pd.DataFrame(data)

In [45]:
df

,Titles,Critics Score,Audience Score,Premiere
0,Mamamoo: My Con the Movie,None,100,"Aug 09, 2023"
1,Hunting for the Hag,None,None,"Aug 09, 2023"
2,Underdog,None,None,"Aug 10, 2023"
3,Jailer,None,84,"Aug 10, 2023"
4,Bholaa Shankar,None,43,"Aug 10, 2023"
...,...,...,...,...
97,Men of Deeds,100,None,"Aug 04, 2023"
98,Growing Up Local,None,None,"Aug 04, 2023"
99,Miss Shetty Mr Polishetty,None,None,"Aug 04, 2023"
100,Rosemary & Sage Turn Back Thyme,None,None,"Aug 04, 2023"


In [46]:
import datetime
import os
fecha_actual = datetime.datetime.now().strftime("%d-%m-%Y")
csv_folder = os.path.join("dataset")
csv_filename = f"On-Theaters-Rotten-Tomatoes-{fecha_actual}.csv"
csv_path = os.path.join(csv_folder, csv_filename)
df.to_csv(csv_path, index=False)

